# DQN Training Experiments for Bowling Atari Environment

**Group Member Name:** Benitha Uwituze Rutagengwa

This notebook loads the Bowling environment and runs **10 training experiments** with different hyperparameter configurations from the CSV file. Each experiment tests 10 different combinations of:
- Learning Rate (lr)
- Gamma (γ) - Discount factor
- Batch Size
- Epsilon parameters (start, end, decay)

## 1. Setup and Imports


In [1]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import ale_py to register ALE namespace
import ale_py


## 2. Load Environment


In [2]:
def make_env(env_id="ALE/Bowling-v5", render_mode=None):
    """Create and wrap the environment"""
    env = gym.make(env_id, render_mode=render_mode)
    env = Monitor(env)
    return env

# Test environment loading
print("Loading Bowling environment...")
test_env = make_env()
print(f"Environment loaded: {test_env}")
print(f"Action space: {test_env.action_space}")
print(f"Observation space: {test_env.observation_space}")

# Test a random action
obs, info = test_env.reset()
print(f"\nObservation shape: {obs.shape}")
print("Environment is ready!")
test_env.close()

Loading Bowling environment...
Environment loaded: <Monitor<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Bowling-v5>>>>>
Action space: Discrete(6)
Observation space: Box(0, 255, (210, 160, 3), uint8)

Observation shape: (210, 160, 3)
Environment is ready!


## 3. Load Hyperparameters from CSV


In [3]:
# Load hyperparameters from CSV
csv_file = "Benitha parameters for experiments.csv"

# Read CSV, skipping the first empty row
df = pd.read_csv(csv_file, skiprows=1, skipinitialspace=True)

# Clean column names (remove spaces and special characters)
df.columns = df.columns.str.strip()

# Remove completely empty rows
df = df.dropna(how='all')

# Display the loaded configurations
print("Loaded hyperparameter configurations:")
print("="*80)
display(df)

# Prepare experiments list
experiments = []
for idx, row in df.iterrows():
    # Check if this row has valid data
    try:
        # Get experiment number - handle both '#' and numeric index
        exp_num = None
        if '#' in row.index:
            exp_num = row['#']
        elif 0 in row.index:
            exp_num = row.iloc[0]
        
        if pd.isna(exp_num) or exp_num == '':
            continue
        
        # Try to convert to int to verify it's a number (not a header)
        exp_num = int(float(exp_num))
        
        # Get learning rate
        lr = None
        if 'lr (Learning Rate)' in row.index:
            lr = row['lr (Learning Rate)']
        elif 'lr' in row.index:
            lr = row['lr']
        
        if pd.isna(lr) or lr == '':
            continue
        
        # Build experiment dictionary
        exp = {
            'num': exp_num,
            'lr': float(row['lr (Learning Rate)']),
            'gamma': float(row['gamma (Discount Factor)']),
            'batch_size': int(row['batch_size']),
            'epsilon_start': float(row['epsilon_start']),
            'epsilon_end': float(row['epsilon_end']),
            'epsilon_decay': float(row['epsilon_decay'])
        }
        experiments.append(exp)
    except (ValueError, TypeError, KeyError) as e:
        # Skip rows that can't be parsed (likely header or invalid rows)
        print(f"  Skipping row {idx}: {e}")
        continue

print(f"\nPrepared {len(experiments)} experiments for training")
if len(experiments) > 0:
    print("\nExperiment configurations:")
    for exp in experiments:
        print(f"  Exp {exp['num']}: lr={exp['lr']}, gamma={exp['gamma']}, batch={exp['batch_size']}, "
              f"eps={exp['epsilon_start']}->{exp['epsilon_end']}, decay={exp['epsilon_decay']}")
else:
    print("\nWARNING: No experiments loaded! Check CSV file format.")
    print("Expected columns: #, lr (Learning Rate), gamma (Discount Factor), batch_size, epsilon_start, epsilon_end, epsilon_decay")

Loaded hyperparameter configurations:


,1,0.001,0.99,32,1.1,0.05,1.00E-05
0,2,0.0005,0.95,64,1,0.10,0.00005
1,3,0.0020,0.97,32,1,0.05,0.00002
2,4,0.0008,0.90,128,1,0.05,0.00010
3,5,0.0015,0.99,64,1,0.10,0.00010
4,6,0.0003,0.95,128,1,0.05,0.00050
5,7,0.0040,0.92,32,1,0.10,0.00002
6,8,0.0012,0.96,64,1,0.05,0.00003
7,9,0.0007,0.93,64,1,0.10,0.00001
8,10,0.0030,0.99,32,1,0.05,0.00004



Prepared 0 experiments for training

Expected columns: #, lr (Learning Rate), gamma (Discount Factor), batch_size, epsilon_start, epsilon_end, epsilon_decay


## 4. Training Function


In [4]:
def convert_epsilon_decay(epsilon_decay):
    """Convert epsilon_decay rate to exploration_fraction"""
    if epsilon_decay < 0.0001:
        return 0.8  # Very slow decay -> decay over most of training
    elif epsilon_decay < 0.0005:
        return 0.5  # Slow decay -> decay over middle portion
    else:
        return 0.2  # Fast decay -> decay over early portion

def train_experiment(exp_config, total_timesteps=50000, policy_type="CnnPolicy"):
    """
    Train a DQN agent with specified hyperparameters
    
    Args:
        exp_config: Dictionary with hyperparameters
        total_timesteps: Total training timesteps
        policy_type: "CnnPolicy" or "MlpPolicy"
    
    Returns:
        model: Trained model
        log_dir: Path to logs
        model_dir: Path to saved model
        training_info: Dictionary with training statistics
    """
    exp_num = exp_config['num']
    
    print(f"EXPERIMENT {exp_num}")
    print(f"Policy: {policy_type}")
    print(f"Learning Rate: {exp_config['lr']}")
    print(f"Gamma: {exp_config['gamma']}")
    print(f"Batch Size: {exp_config['batch_size']}")
    print(f"Epsilon: {exp_config['epsilon_start']} -> {exp_config['epsilon_end']}")
    
    # Convert epsilon_decay to exploration_fraction
    exploration_fraction = convert_epsilon_decay(exp_config['epsilon_decay'])
    print(f"Epsilon Decay Rate: {exp_config['epsilon_decay']} -> Exploration Fraction: {exploration_fraction}")
    print(f"Total Timesteps: {total_timesteps}")
    
    # Create directories
    experiment_name = f"benitha_exp_{exp_num}"
    log_dir = f"benitha_logs/{experiment_name}_{policy_type}"
    model_dir = f"benitha_models/{experiment_name}_{policy_type}"
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    
    # Create environments
    env = DummyVecEnv([lambda: make_env()])
    eval_env = DummyVecEnv([lambda: make_env()])
    
    # Initialize DQN agent
    # Use smaller buffer_size for Atari to avoid memory issues (default is 1M which is too large)
    model = DQN(
        policy_type,
        env,
        learning_rate=exp_config['lr'],
        gamma=exp_config['gamma'],
        batch_size=exp_config['batch_size'],
        buffer_size=5000,  # Reduced from default 1M to save memory (9.4GB instead of 94GB)
        exploration_initial_eps=exp_config['epsilon_start'],
        exploration_final_eps=exp_config['epsilon_end'],
        exploration_fraction=exploration_fraction,
        verbose=1,
        tensorboard_log=log_dir,
        device="auto"
    )
    
    # Setup callbacks
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=model_dir,
        log_path=log_dir,
        eval_freq=5000,
        deterministic=True,
        render=False
    )
    
    checkpoint_callback = CheckpointCallback(
        save_freq=10000,
        save_path=model_dir,
        name_prefix="dqn_checkpoint"
    )
    
    # Train the model
    print("Starting training...")
    model.learn(
        total_timesteps=total_timesteps,
        callback=[eval_callback, checkpoint_callback],
        progress_bar=True
    )
    
    # Save final model
    final_model_path = f"{model_dir}/benitha_dqn_model.zip"
    model.save(final_model_path)
    print(f"\nModel saved to: {final_model_path}")
    
    # Extract training statistics from monitor
    training_info = {
        'experiment': exp_num,
        'policy': policy_type,
        'final_timesteps': total_timesteps
    }
    
    # Try to get evaluation results
    if hasattr(eval_callback, 'best_mean_reward'):
        training_info['best_mean_reward'] = eval_callback.best_mean_reward
        training_info['n_eval_episodes'] = eval_callback.n_eval_episodes
    
    return model, log_dir, model_dir, training_info

## 6. Run Individual Experiments



In [5]:
# Configuration for all experiments
TOTAL_TIMESTEPS = 30000  # Adjust based on available time
POLICY_TYPE = "CnnPolicy"  # Use "MlpPolicy" for comparison

# Load existing results if available (prevents re-running completed experiments)
if os.path.exists('benitha_experiment_results.csv'):
    try:
        existing_df = pd.read_csv('benitha_experiment_results.csv')
        all_results = existing_df.to_dict('records')
        print(f"Loaded {len(all_results)} existing experiment results from CSV")
        successful = [r for r in all_results if r.get('status') == 'success']
        print(f"  - {len(successful)} successful experiments")
        print(f"  - {len(all_results) - len(successful)} failed/incomplete experiments")
    except Exception as e:
        print(f"Could not load existing results: {e}")
        all_results = []
        all_training_info = []
else:
    all_results = []
    all_training_info = []

print(f"\nConfiguration:")
print(f"  Total timesteps per experiment: {TOTAL_TIMESTEPS}")
print(f"  Policy type: {POLICY_TYPE}")
print(f"  Total experiments to run: 10")
print(f"  Estimated total time: ~{10 * TOTAL_TIMESTEPS / 1000:.0f} minutes")
print(f"\nNote: Completed experiments will be skipped automatically.")


Loaded 7 existing experiment results from CSV
  - 7 successful experiments
  - 0 failed/incomplete experiments

Configuration:
  Total timesteps per experiment: 30000
  Policy type: CnnPolicy
  Total experiments to run: 10
  Estimated total time: ~300 minutes

Note: Completed experiments will be skipped automatically.


## Experiment 1


In [6]:
# Experiment 1: lr=0.0003, gamma=0.95, batch=32
# Epsilon: 1.0 -> 0.01, decay=5e-05

# Check if already completed
exp_1_completed = any(r.get('experiment') == 1 and r.get('status') == 'success' for r in all_results)

if exp_1_completed:
    print("Experiment 1 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 1,
            'lr': 0.0010,
            'gamma': 0.99,
            'batch_size': 32,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 1e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 1,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        

        print(f"Experiment 1 completed in {duration:.2f} minutes")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        
    except Exception as e:
        print(f"\nError in experiment 1: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 1,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 1 already completed. Skipping.


## Experiment 2


In [7]:
# Check if already completed
exp_2_completed = any(r.get('experiment') == 2 and r.get('status') == 'success' for r in all_results)

if exp_2_completed:
    print("Experiment 2 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 2,
            'lr': 0.0005,
            'gamma': 0.95,
            'batch_size': 64,
            'epsilon_start': 1.0,
            'epsilon_end': 0.1,
            'epsilon_decay': 5e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 2,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 2 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 2: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 2,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 2 already completed. Skipping.


## Experiment 3


In [8]:
# Check if already completed
exp_3_completed = any(r.get('experiment') == 3 and r.get('status') == 'success' for r in all_results)

if exp_3_completed:
    print("Experiment 3 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 3,
            'lr': 0.0020,
            'gamma': 0.97,
            'batch_size': 32,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 2e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 3,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 3 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 3: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 3,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 3 already completed. Skipping.


## Experiment 4


In [9]:

# Check if already completed
exp_4_completed = any(r.get('experiment') == 4 and r.get('status') == 'success' for r in all_results)

if exp_4_completed:
    print("Experiment 4 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 4,
            'lr': 0.0008,
            'gamma': 0.90,
            'batch_size': 128,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 1e-04
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 4,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 4 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 4: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 4,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 4 already completed. Skipping.


## Experiment 5


In [10]:
# Experiment 5: lr=0.0001, gamma=0.99, batch=64
# Epsilon: 1.0 -> 0.05, decay=0.0001

# Check if already completed
exp_5_completed = any(r.get('experiment') == 5 and r.get('status') == 'success' for r in all_results)

if exp_5_completed:
    print("Experiment 5 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 5,
            'lr': 0.0015,
            'gamma': 0.99,
            'batch_size': 64,
            'epsilon_start': 1.0,
            'epsilon_end': 0.1,
            'epsilon_decay': 1e-04
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 5,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 5 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 5: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 5,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 5 already completed. Skipping.


## Experiment 6


In [11]:
# Experiment 6: lr=0.0015, gamma=0.92, batch=128
# Epsilon: 1.0 -> 0.1, decay=5e-05

# Check if already completed
exp_6_completed = any(r.get('experiment') == 6 and r.get('status') == 'success' for r in all_results)

if exp_6_completed:
    print("Experiment 6 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 6,
            'lr': 0.0003,
            'gamma': 0.95,
            'batch_size': 128,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 5e-04
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 6,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 6 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 6: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 6,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 6 already completed. Skipping.


## Experiment 7


In [12]:
# Experiment 7: lr=0.0007, gamma=0.98, batch=32
# Epsilon: 1.0 -> 0.01, decay=2e-05

# Check if already completed
exp_7_completed = any(r.get('experiment') == 7 and r.get('status') == 'success' for r in all_results)

if exp_7_completed:
    print("Experiment 7 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 7,
            'lr': 0.0040,
            'gamma': 0.92,
            'batch_size': 32,
            'epsilon_start': 1.0,
            'epsilon_end': 0.1,
            'epsilon_decay': 2e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 7,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 7 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 7: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 7,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

Experiment 7 already completed. Skipping.


## Experiment 8


In [13]:
# Experiment 8: lr=0.002, gamma=0.96, batch=64
# Epsilon: 1.0 -> 0.05, decay=0.0001

# Check if already completed
exp_8_completed = any(r.get('experiment') == 8 and r.get('status') == 'success' for r in all_results)

if exp_8_completed:
    print("Experiment 8 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 8,
            'lr': 0.0012,
            'gamma': 0.96,
            'batch_size': 64,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 3e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 8,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 8 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 8: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 8,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

EXPERIMENT 8
Policy: CnnPolicy
Learning Rate: 0.0012
Gamma: 0.96
Batch Size: 64
Epsilon: 1.0 -> 0.05
Epsilon Decay Rate: 3e-05 -> Exploration Fraction: 0.8
Total Timesteps: 30000
Using cpu device
Wrapping the env in a VecTransposeImage.
Starting training...
Logging to benitha_logs/benitha_exp_8_CnnPolicy\DQN_2


Eval num_timesteps=5000, episode_reward=0.00 +/- 0.00

Episode length: 2002.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2e+03    |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.802    |
| time/               |          |
|    total_timesteps  | 5000     |
| train/              |          |
|    learning_rate    | 0.0012   |
|    loss             | 3.66e-05 |
|    n_updates        | 1224     |
----------------------------------


New best mean reward!

...

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 25000    |
| train/              |          |
|    learning_rate    | 0.0012   |
|    loss             | 1.05e-05 |
|    n_updates        | 6224     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.15e+03 |
|    ep_rew_mean      | 26.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4        |
|    time_elapsed     | 5651     |
|    total_timesteps  | 25778    |
| train/              |          |
|    learning_rate    | 0.0012   |
|    loss             | 0.00718  |
|    n_updates        | 6419     |
----------------------------------


Eval num_timesteps=30000, episode_reward=0.00 +/- 0.00

Episode length: 27000.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 30000    |
| train/              |          |
|    learning_rate    | 0.0012   |
|    loss             | 8.31e-07 |
|    n_updates        | 7474     |
----------------------------------



Model saved to: benitha_models/benitha_exp_8_CnnPolicy/benitha_dqn_model.zip
Experiment 8 completed in 112.72 minutes
  Model saved to: benitha_models/benitha_exp_8_CnnPolicy/benitha_dqn_model.zip
  Results saved to: benitha_experiment_results.csv
  Best Mean Reward: 30.00


## Experiment 9


In [14]:
# Experiment 9: lr=0.0004, gamma=0.93, batch=128
# Epsilon: 1.0 -> 0.1, decay=0.0005

# Check if already completed
exp_9_completed = any(r.get('experiment') == 9 and r.get('status') == 'success' for r in all_results)

if exp_9_completed:
    print("Experiment 9 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 9,
            'lr': 0.0007,
            'gamma': 0.93,
            'batch_size': 64,
            'epsilon_start': 1.0,
            'epsilon_end': 0.1,
            'epsilon_decay': 1e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 9,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 9 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 9: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 9,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

EXPERIMENT 9
Policy: CnnPolicy
Learning Rate: 0.0007
Gamma: 0.93
Batch Size: 64
Epsilon: 1.0 -> 0.1
Epsilon Decay Rate: 1e-05 -> Exploration Fraction: 0.8
Total Timesteps: 30000
Using cpu device
Wrapping the env in a VecTransposeImage.
Starting training...
Logging to benitha_logs/benitha_exp_9_CnnPolicy\DQN_1


Eval num_timesteps=5000, episode_reward=0.00 +/- 0.00

Episode length: 27000.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.813    |
| time/               |          |
|    total_timesteps  | 5000     |
| train/              |          |
|    learning_rate    | 0.0007   |
|    loss             | 9.6e-06  |
|    n_updates        | 1224     |
----------------------------------


New best mean reward!

...

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.1      |
| time/               |          |
|    total_timesteps  | 25000    |
| train/              |          |
|    learning_rate    | 0.0007   |
|    loss             | 5.42e-06 |
|    n_updates        | 6224     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.13e+03 |
|    ep_rew_mean      | 23       |
|    exploration_rate | 0.1      |
| time/               |          |
|    episodes         | 12       |
|    fps              | 7        |
|    time_elapsed     | 3584     |
|    total_timesteps  | 25599    |
| train/              |          |
|    learning_rate    | 0.0007   |
|    loss             | 1.69e-05 |
|    n_updates        | 6374     |
----------------------------------


Eval num_timesteps=30000, episode_reward=0.00 +/- 0.00

Episode length: 27000.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.1      |
| time/               |          |
|    total_timesteps  | 30000    |
| train/              |          |
|    learning_rate    | 0.0007   |
|    loss             | 0.00734  |
|    n_updates        | 7474     |
----------------------------------



Model saved to: benitha_models/benitha_exp_9_CnnPolicy/benitha_dqn_model.zip
Experiment 9 completed in 78.38 minutes
  Model saved to: benitha_models/benitha_exp_9_CnnPolicy/benitha_dqn_model.zip
  Results saved to: benitha_experiment_results.csv
  Best Mean Reward: 30.00


## Experiment 10


In [15]:
# Experiment 10: lr=0.0035, gamma=0.99, batch=32
# Epsilon: 1.0 -> 0.02, decay=3e-05

# Check if already completed
exp_10_completed = any(r.get('experiment') == 10 and r.get('status') == 'success' for r in all_results)

if exp_10_completed:
    print("Experiment 10 already completed. Skipping.")
else:
    try:
        start_time = datetime.now()
        
        exp_config = {
            'num': 10,
            'lr': 0.0030,
            'gamma': 0.99,
            'batch_size': 32,
            'epsilon_start': 1.0,
            'epsilon_end': 0.05,
            'epsilon_decay': 4e-05
        }
        
        model, log_dir, model_dir, training_info = train_experiment(
            exp_config,
            total_timesteps=TOTAL_TIMESTEPS,
            policy_type=POLICY_TYPE
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60  # in minutes
        
        result = {
            'experiment': 10,
            'status': 'success',
            'duration_minutes': duration,
            'log_dir': log_dir,
            'model_dir': model_dir,
            'best_mean_reward': training_info.get('best_mean_reward', None),
            **exp_config
        }
        
        all_results.append(result)

        # Save results to CSV after each experiment
        results_df = pd.DataFrame(all_results)
        results_df.to_csv('benitha_experiment_results.csv', index=False)
        
        print(f"Experiment 10 completed in {duration:.2f} minutes")
        print(f"  Model saved to: {model_dir}/benitha_dqn_model.zip")
        print(f"  Results saved to: benitha_experiment_results.csv")
        if training_info.get('best_mean_reward') is not None:
            print(f"  Best Mean Reward: {training_info['best_mean_reward']:.2f}")
        
    except Exception as e:
        print(f"\nError in experiment 10: {e}")
        import traceback
        traceback.print_exc()
        result = {
            'experiment': 10,
            'status': 'failed',
            'error': str(e),
            **exp_config
        }
        all_results.append(result)

EXPERIMENT 10
Policy: CnnPolicy
Learning Rate: 0.003
Gamma: 0.99
Batch Size: 32
Epsilon: 1.0 -> 0.05
Epsilon Decay Rate: 4e-05 -> Exploration Fraction: 0.8
Total Timesteps: 30000
Using cpu device
Wrapping the env in a VecTransposeImage.
Starting training...
Logging to benitha_logs/benitha_exp_10_CnnPolicy\DQN_1


Eval num_timesteps=5000, episode_reward=0.00 +/- 0.00

Episode length: 27000.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.802    |
| time/               |          |
|    total_timesteps  | 5000     |
| train/              |          |
|    learning_rate    | 0.003    |
|    loss             | 1.16e-05 |
|    n_updates        | 1224     |
----------------------------------


New best mean reward!

...

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.7e+04  |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 25000    |
| train/              |          |
|    learning_rate    | 0.003    |
|    loss             | 1.33e-05 |
|    n_updates        | 6224     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.14e+03 |
|    ep_rew_mean      | 23.1     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5        |
|    time_elapsed     | 4974     |
|    total_timesteps  | 25723    |
| train/              |          |
|    learning_rate    | 0.003    |
|    loss             | 0.0779   |
|    n_updates        | 6405     |
----------------------------------


Eval num_timesteps=30000, episode_reward=0.00 +/- 0.00

Episode length: 2002.00 +/- 0.00

----------------------------------
| eval/               |          |
|    mean_ep_length   | 2e+03    |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 30000    |
| train/              |          |
|    learning_rate    | 0.003    |
|    loss             | 0.0761   |
|    n_updates        | 7474     |
----------------------------------



Model saved to: benitha_models/benitha_exp_10_CnnPolicy/benitha_dqn_model.zip
Experiment 10 completed in 93.75 minutes
  Model saved to: benitha_models/benitha_exp_10_CnnPolicy/benitha_dqn_model.zip
  Results saved to: benitha_experiment_results.csv
  Best Mean Reward: 30.00


## 6.5. Experiments Summary


In [16]:
# Summary of all completed experiments
# Load results from CSV if available
if os.path.exists('benitha_experiment_results.csv'):
    try:
        results_df = pd.read_csv('benitha_experiment_results.csv')
        all_results = results_df.to_dict('records')
    except:
        all_results = []
elif 'all_results' in globals():
    # Use in-memory results
    pass
else:
    all_results = []

if len(all_results) > 0:
    successful = [r for r in all_results if r.get('status') == 'success']
    failed = [r for r in all_results if r.get('status') == 'failed']
    
    print(f"EXPERIMENTS SUMMARY")
    print(f"\nCompleted: {len(successful)}/10")
    print(f"Failed: {len(failed)}/10")
    
    if len(successful) > 0:
        print(f"\nSuccessful experiments: {sorted([r['experiment'] for r in successful])}")
        # Show rewards if available
        rewards = [r.get('best_mean_reward') for r in successful if pd.notna(r.get('best_mean_reward'))]
        if rewards:
            print(f"\nReward Statistics:")
            print(f"  Best: {max(rewards):.2f}")
            print(f"  Average: {sum(rewards)/len(rewards):.2f}")
    if len(failed) > 0:
        print(f"\nFailed experiments: {sorted([r['experiment'] for r in failed])}")
    
    # Save results to CSV for persistence
    results_df = pd.DataFrame(all_results)
    results_df.to_csv('benitha_experiment_results.csv', index=False)
    print(f"\nResults saved to benitha_experiment_results.csv")
else:
    print("No experiments completed yet. Run the experiment cells above.")

EXPERIMENTS SUMMARY

Completed: 10/10
Failed: 0/10

Successful experiments: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Reward Statistics:
  Best: 35.40
  Average: 21.34

Results saved to benitha_experiment_results.csv


## 7. Compare MLPPolicy vs CNNPolicy (Required)


In [17]:
# Compare policies using the first experiment configuration
# This is REQUIRED for the assignment - compare MLPPolicy vs CNNPolicy
if len(experiments) > 0:
    comparison_exp = experiments[0].copy()
    
    print("COMPARING MLPPolicy vs CNNPolicy")
    print(f"Using experiment {comparison_exp['num']} configuration")
    print("This comparison is required for the assignment.")
    
    comparison_results = {}
    
    for policy in ["MlpPolicy", "CnnPolicy"]:
        print(f"Training with {policy}")
        
        try:
            model, log_dir, model_dir, training_info = train_experiment(
                comparison_exp,
                total_timesteps=20000,  # Reduced for comparison
                policy_type=policy
            )
            
            comparison_results[policy] = {
                'status': 'success',
                'log_dir': log_dir,
                'model_dir': model_dir,
                'best_mean_reward': training_info.get('best_mean_reward', None)
            }
            
            if training_info.get('best_mean_reward') is not None:
                print(f"\n{policy} Best Mean Reward: {training_info['best_mean_reward']:.2f}")
            
        except Exception as e:
            print(f"Error with {policy}: {e}")
            import traceback
            traceback.print_exc()
            comparison_results[policy] = {
                'status': 'failed',
                'error': str(e)
            }
    
    print("POLICY COMPARISON COMPLETE")
    print("\nResults Summary:")
    for policy, result in comparison_results.items():
        if result['status'] == 'success':
            reward_info = f"Best Reward: {result.get('best_mean_reward', 'N/A')}" if result.get('best_mean_reward') else "Reward: N/A"
            print(f"  {policy}: {reward_info}")
            print(f"    Logs: {result['log_dir']}")
    print("\nView TensorBoard logs to compare performance:")
    print("  tensorboard --logdir benitha_logs/")


In [18]:
# Create results DataFrame
if all_results:
    results_df = pd.DataFrame(all_results)
    
    print("EXPERIMENT RESULTS SUMMARY")
    
    # Display key columns
    display_cols = ['experiment', 'status', 'lr', 'gamma', 'batch_size', 
                    'epsilon_start', 'epsilon_end', 'epsilon_decay', 
                    'best_mean_reward', 'duration_minutes']
    available_cols = [col for col in display_cols if col in results_df.columns]
    display(results_df[available_cols])
    
    # Summary statistics
    successful = results_df[results_df['status'] == 'success']
    if len(successful) > 0:
        print("SUMMARY STATISTICS")
        print(f"Successful experiments: {len(successful)}")
        print(f"Average training time: {successful['duration_minutes'].mean():.2f} minutes")
        print(f"Total training time: {successful['duration_minutes'].sum():.2f} minutes")
        
        if 'best_mean_reward' in successful.columns:
            valid_rewards = successful['best_mean_reward'].dropna()
            if len(valid_rewards) > 0:
                print(f"\nReward Statistics:")
                print(f"  Best reward: {valid_rewards.max():.2f}")
                print(f"  Worst reward: {valid_rewards.min():.2f}")
                print(f"  Average reward: {valid_rewards.mean():.2f}")
                print(f"  Median reward: {valid_rewards.median():.2f}")
    
    failed = results_df[results_df['status'] == 'failed']
    if len(failed) > 0:
        print(f"\nFailed experiments: {len(failed)}")
        for idx, row in failed.iterrows():
            print(f"  Experiment {row['experiment']}: {row.get('error', 'Unknown error')}")
    
    # Save results to CSV
    results_df.to_csv('benitha_experiment_results.csv', index=False)
    print("\n Results saved to benitha_experiment_results.csv")
else:
    print("No results to display. Run experiments first.")

EXPERIMENT RESULTS SUMMARY


,experiment,status,lr,gamma,batch_size,epsilon_start,epsilon_end,epsilon_decay,best_mean_reward,duration_minutes
0,1,success,0.0010,0.99,32,1.0,0.05,0.00001,30.0,80.809031
1,2,success,0.0005,0.95,64,1.0,0.10,0.00005,35.4,129.450441
2,3,success,0.0020,0.97,32,1.0,0.05,0.00002,0.0,94.039755
3,4,success,0.0008,0.90,128,1.0,0.05,0.00010,0.0,89.919371
4,5,success,0.0015,0.99,64,1.0,0.10,0.00010,0.0,132.100118
5,6,success,0.0003,0.95,128,1.0,0.05,0.00050,28.0,120.271615
6,7,success,0.0040,0.92,32,1.0,0.10,0.00002,30.0,105.477129
7,8,success,0.0012,0.96,64,1.0,0.05,0.00003,30.0,112.718469
8,9,success,0.0007,0.93,64,1.0,0.10,0.00001,30.0,78.381118
9,10,success,0.0030,0.99,32,1.0,0.05,0.00004,30.0,93.752422


SUMMARY STATISTICS
Successful experiments: 10
Average training time: 103.69 minutes
Total training time: 1036.92 minutes

Reward Statistics:
  Best reward: 35.40
  Worst reward: 0.00
  Average reward: 21.34
  Median reward: 30.00

 Results saved to benitha_experiment_results.csv


In [19]:
# Function to analyze and generate observed behaviors
def analyze_experiment_behavior(row, all_successful):
    """
    Automatically analyze experiment results and generate behavior description
    Args:
        row: Single experiment row from results DataFrame
        all_successful: All successful experiments for comparison
    Returns:
        behavior_description: String describing observed behavior
    """
    behaviors = []
    
    # Get reward information
    best_reward = row.get('best_mean_reward')
    if pd.notna(best_reward):
        # Compare with other experiments
        all_rewards = all_successful['best_mean_reward'].dropna()
        if len(all_rewards) > 0:
            reward_percentile = (all_rewards <= best_reward).sum() / len(all_rewards) * 100
            if reward_percentile >= 80:
                behaviors.append(f"Excellent performance (top {100-int(reward_percentile)}%)")
            elif reward_percentile >= 60:
                behaviors.append(f"Good performance (above average)")
            elif reward_percentile >= 40:
                behaviors.append(f"Moderate performance (average)")
            else:
                behaviors.append(f"Below average performance")
            behaviors.append(f"Best reward: {best_reward:.2f}")
        else:
            behaviors.append(f"Best reward: {best_reward:.2f}")
    
    # Analyze learning rate
    lr = row['lr']
    if lr > 0.002:
        behaviors.append("High LR - may cause instability")
    elif lr < 0.0002:
        behaviors.append("Low LR - slower but stable learning")
    else:
        behaviors.append("Moderate LR - balanced learning")
    
    # Analyze gamma (discount factor)
    gamma = row['gamma']
    if gamma >= 0.98:
        behaviors.append("High gamma - long-term focus")
    elif gamma <= 0.92:
        behaviors.append("Low gamma - short-term focus")
    else:
        behaviors.append("Moderate gamma - balanced")
    
    # Analyze batch size
    batch = row['batch_size']
    if batch >= 128:
        behaviors.append("Large batch - stable gradients")
    elif batch <= 32:
        behaviors.append("Small batch - faster updates")
    else:
        behaviors.append("Medium batch - balanced")
    
    # Analyze epsilon decay
    eps_decay = row['epsilon_decay']
    if eps_decay < 0.0001:
        behaviors.append("Slow exploration decay - more exploration")
    elif eps_decay > 0.0003:
        behaviors.append("Fast exploration decay - quick exploitation")
    else:
        behaviors.append("Moderate exploration decay")
    
    # Analyze training duration (if available)
    if 'duration_minutes' in row and pd.notna(row['duration_minutes']):
        duration = row['duration_minutes']
        avg_duration = all_successful['duration_minutes'].mean()
        if duration < avg_duration * 0.9:
            behaviors.append("Faster training")
        elif duration > avg_duration * 1.1:
            behaviors.append("Slower training")
    
    # Combine all behaviors
    if behaviors:
        return ". ".join(behaviors) + "."
    else:
        return "Training completed. Review TensorBoard for detailed metrics."

# Generate hyperparameter table for documentation
def create_hyperparameter_table(results_df, auto_fill=True):
    """Create formatted table for README documentation with behavior analysis"""
    if results_df is None or len(results_df) == 0:
        print("No results available. Run experiments first.")
        return
    
    successful = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful) == 0:
        print("No successful experiments to document.")
        return
    
    print("="*120)
    print("HYPERPARAMETER TUNING TABLE - COPY TO README.md")
    if auto_fill:
        print("(Observed Behaviors generated from training results)")
    print("="*120)
    print("\n| Experiment # | Learning Rate | Gamma | Batch Size | Epsilon Start | Epsilon End | Epsilon Decay | Observed Behavior |")
    print("|--------------|---------------|-------|------------|---------------|-------------|---------------|-------------------|")
    
    for idx, row in successful.iterrows():
        exp_num = int(row['experiment'])
        lr = row['lr']
        gamma = row['gamma']
        batch = int(row['batch_size'])
        eps_start = row['epsilon_start']
        eps_end = row['epsilon_end']
        eps_decay = row['epsilon_decay']
        
        # Generate automatic behavior description
        if auto_fill:
            behavior = analyze_experiment_behavior(row, successful)
        else:
            # Get reward info if available
            reward_info = ""
            if pd.notna(row.get('best_mean_reward')):
                reward_info = f"Best reward: {row['best_mean_reward']:.2f}. "
            behavior = f"{reward_info}[DESCRIBE: Convergence speed, stability, final performance, reward trends, episode lengths]"
        
        # Truncate behavior if too long for table
        if len(behavior) > 100:
            behavior = behavior[:97] + "..."
        
        print(f"| {exp_num} | {lr} | {gamma} | {batch} | {eps_start} | {eps_end} | {eps_decay} | {behavior} |")
    
    if auto_fill:
        print("\nNOTE: Behaviors are generated from training metrics.")
        print("You can enhance these descriptions by:")
        print("1. Reviewing TensorBoard logs for detailed reward trends")
        print("2. Observing episode lengths and convergence patterns")
        print("3. Comparing with other experiments")
        print("4. Adding specific observations about stability and learning speed")
    else:
        print("\nINSTRUCTIONS:")
        print("1. Review TensorBoard logs for each experiment")
        print("2. Note the reward trends (increasing, stable, fluctuating)")
        print("3. Observe episode lengths over time")
        print("4. Check convergence speed (how quickly rewards improve)")
        print("5. Note any stability issues (erratic behavior)")
        print("6. Record final performance metrics")
        print("7. Fill in the 'Observed Behavior' column with your findings")
        print("8. Copy this table to your README.md file")

# Generate the table with behavior analysis
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\nGenerating hyperparameter table with behavior analysis...\n")
    create_hyperparameter_table(results_df, auto_fill=True)
    
    # Also save detailed analysis to a file
    print("DETAILED EXPERIMENT ANALYSIS")
    successful = results_df[results_df['status'] == 'success'].copy()
    if len(successful) > 0 and 'best_mean_reward' in successful.columns:
        valid_rewards = successful['best_mean_reward'].dropna()
        if len(valid_rewards) > 0:
            print(f"\nBest Performing Experiment: {successful.loc[valid_rewards.idxmax(), 'experiment']}")
            print(f"  Reward: {valid_rewards.max():.2f}")
            print(f"\nWorst Performing Experiment: {successful.loc[valid_rewards.idxmin(), 'experiment']}")
            print(f"  Reward: {valid_rewards.min():.2f}")
            print(f"\nAverage Reward Across All Experiments: {valid_rewards.mean():.2f}")
            print(f"Standard Deviation: {valid_rewards.std():.2f}")
else:
    print("Run experiments first to generate the table.")


Generating hyperparameter table with behavior analysis...

HYPERPARAMETER TUNING TABLE - COPY TO README.md
(Observed Behaviors generated from training results)

| Experiment # | Learning Rate | Gamma | Batch Size | Epsilon Start | Epsilon End | Epsilon Decay | Observed Behavior |
|--------------|---------------|-------|------------|---------------|-------------|---------------|-------------------|
| 1 | 0.001 | 0.99 | 32 | 1.0 | 0.05 | 1e-05 | Excellent performance (top 10%). Best reward: 30.00. Moderate LR - balanced learning. High gamma ... |
| 2 | 0.0005 | 0.95 | 64 | 1.0 | 0.1 | 5e-05 | Excellent performance (top 0%). Best reward: 35.40. Moderate LR - balanced learning. Moderate gam... |
| 3 | 0.002 | 0.97 | 32 | 1.0 | 0.05 | 2e-05 | Below average performance. Best reward: 0.00. Moderate LR - balanced learning. Moderate gamma - b... |
| 4 | 0.0008 | 0.9 | 128 | 1.0 | 0.05 | 0.0001 | Below average performance. Best reward: 0.00. Moderate LR - balanced learning. Low gamma - short-..

In [20]:
# Instructions for viewing TensorBoard
print("To view training progress with TensorBoard, run in terminal:")
print("\n tensorboard --logdir benitha_logs/")
print("\nThen open http://localhost:6006 in your browser")
print("\nOr use TensorBoard in Jupyter:")
print(" %load_ext tensorboard")
print(" %tensorboard --logdir benitha_logs/")


To view training progress with TensorBoard, run in terminal:

 tensorboard --logdir benitha_logs/

Then open http://localhost:6006 in your browser

Or use TensorBoard in Jupyter:
 %load_ext tensorboard
 %tensorboard --logdir benitha_logs/


## 11. Load and Test Trained Model


In [21]:
# Load a trained model and test it
def test_model(model_path, num_episodes=3, render=False):
    """Test a trained model"""
    
    print(f"Loading model from: {model_path}")
    
    # Create environment
    env = make_env(render_mode="human" if render else None)
    
    # Load model
    try:
        model = DQN.load(model_path, env=env)
        print(" Model loaded successfully")
    except Exception as e:
        print(f" Error loading model: {e}")
        env.close()
        return
    
    # Set to greedy policy (no exploration)
    model.exploration_rate = 0.0
    
    # Test episodes
    total_rewards = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        
        while not (done or truncated):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            
            if render:
                env.render()
        
        total_rewards.append(episode_reward)
        print(f"Episode {episode + 1}: Reward = {episode_reward:.2f}")
    
    env.close()
    
    avg_reward = np.mean(total_rewards)
    print(f"\nAverage reward over {num_episodes} episodes: {avg_reward:.2f}")
    
    return avg_reward

# Test the best model from first experiment (if available)
if len(experiments) > 0:
    test_model_path = f"benitha_models/benitha_exp_{experiments[0]['num']}_CnnPolicy/best_model.zip"
    if os.path.exists(test_model_path):
        print("Testing trained model...")
        test_model(test_model_path, num_episodes=3, render=False)
    else:
        print(f"Model not found at {test_model_path}")
        print("Train a model first.")

In [22]:
# Select and save the best model as benitha_dqn_model.zip for use in play.py
# This is REQUIRED for the assignment

# Load results from CSV if available (works even after notebook restart)
if os.path.exists('benitha_experiment_results.csv'):
    try:
        results_df = pd.read_csv('benitha_experiment_results.csv')
        all_results = results_df.to_dict('records')
        print(f"Loaded {len(all_results)} experiment results from CSV")
    except Exception as e:
        print(f"Error loading results: {e}")
        all_results = []
elif 'all_results' not in globals():
    all_results = []

if all_results:
    results_df = pd.DataFrame(all_results)
    successful = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful) > 0:
        # Find the best performing experiment based on best_mean_reward
        if 'best_mean_reward' in successful.columns:
            valid_rewards = successful['best_mean_reward'].dropna()
            if len(valid_rewards) > 0:
                best_exp_idx = valid_rewards.idxmax()
                best_exp = successful.loc[best_exp_idx]
                best_exp_num = int(best_exp['experiment'])
                
                # Path to best model
                best_model_path = f"benitha_models/benitha_exp_{best_exp_num}_CnnPolicy/best_model.zip"
                final_model_path = f"benitha_models/benitha_exp_{best_exp_num}_CnnPolicy/benitha_dqn_model.zip"
                
                # Try best_model first, then final model
                if os.path.exists(best_model_path):
                    import shutil
                    shutil.copy(best_model_path, "benitha_dqn_model.zip")
                    print(f" Best model copied from Experiment {best_exp_num}")
                    print(f"  Source: {best_model_path}")
                    print(f"  Destination: benitha_dqn_model.zip")
                    print(f"  Best Reward: {best_exp['best_mean_reward']:.2f}")
                elif os.path.exists(final_model_path):
                    import shutil
                    shutil.copy(final_model_path, "benitha_dqn_model.zip")
                    print(f" Final model copied from Experiment {best_exp_num}")
                    print(f"  Source: {final_model_path}")
                    print(f"  Destination: benitha_dqn_model.zip")
                    print(f"  Best Reward: {best_exp['best_mean_reward']:.2f}")
                else:
                    print(f" Model files not found for Experiment {best_exp_num}")
                    print(f"  Expected: {best_model_path} or {final_model_path}")
                    print(f"  Please check if training completed successfully.")
            else:
                print("No reward data available. Using first successful experiment.")
                first_exp = successful.iloc[0]
                first_exp_num = int(first_exp['experiment'])
                model_path = f"benitha_models/benitha_exp_{first_exp_num}_CnnPolicy/benitha_dqn_model.zip"
                if os.path.exists(model_path):
                    import shutil
                    shutil.copy(model_path, "benitha_dqn_model.zip")
                    print(f" Model copied from Experiment {first_exp_num}")
                else:
                    print(f" Model not found at {model_path}")
        else:
            print("No reward data available. Using first successful experiment.")
            first_exp = successful.iloc[0]
            first_exp_num = int(first_exp['experiment'])
            model_path = f"benitha_models/benitha_exp_{first_exp_num}_CnnPolicy/benitha_dqn_model.zip"
            if os.path.exists(model_path):
                import shutil
                shutil.copy(model_path, "benitha_dqn_model.zip")
                print(f" Model copied from Experiment {first_exp_num}")
            else:
                print(f" Model not found at {model_path}")
        
        # Verify the file exists
        if os.path.exists("benitha_dqn_model.zip"):
            file_size = os.path.getsize("benitha_dqn_model.zip") / (1024 * 1024)  # MB
            print(f"\n benitha_dqn_model.zip is ready for play.py")
            print(f"  File size: {file_size:.2f} MB")
        else:
            print("\n benitha_dqn_model.zip was not created. Please check model paths.")
    else:
        print("No successful experiments found. Run experiments first.")
else:
    print("No results available. Run experiments first.")

Loaded 10 experiment results from CSV
 Best model copied from Experiment 2
  Source: benitha_models/benitha_exp_2_CnnPolicy/best_model.zip
  Destination: benitha_dqn_model.zip
  Best Reward: 35.40

 benitha_dqn_model.zip is ready for play.py
  File size: 178.01 MB
